In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re

In [16]:
df_modPar = pd.read_csv('/Users/andreas/phys/master/GRB/xspec/ModelPar_Analysis/modelPar.csv', header=0, index_col=0, sep='&')
df_modPar.columns = df_modPar.columns.str.strip()
df_modPar = df_modPar.drop(columns = ['chi2/dof','C_A','C_F'])


# Define a function to extract the three values from each parameter cell
def extract_values(cell):
    # Convert the cell to a string
    cell = str(cell)
    # Remove any special characters
    cell = re.sub(r'[\$\^_{}\\*]', '', cell)
    # Extract the three values
    values = re.findall(r'[-+]?[0-9]*\.?[0-9]+', cell)
    if len(values) == 3:
        lower, value, upper = map(float, values)
    else:
        lower = value = upper = float('nan')
    return lower, value, upper


param_cols = ['alpha','beta','Epeak']
# Loop over the parameter columns
for col in param_cols:
    # Create new columns for lower, value, and upper
    df_modPar[col+'_lower'] = np.nan
    df_modPar[col+'_value'] = np.nan
    df_modPar[col+'_upper'] = np.nan
    # Apply the extract_values function to each cell in the column
    for idx, cell in df_modPar[col].items():
        lower, value, upper = extract_values(cell)
        df_modPar.at[idx, col+'_lower'] = lower
        df_modPar.at[idx, col+'_value'] = value
        df_modPar.at[idx, col+'_upper'] = upper
    # Remove the original parameter column
    df_modPar.drop(col, axis=1, inplace=True)
df_modPar

,Int,alpha_lower,alpha_value,alpha_upper,beta_lower,beta_value,beta_upper,Epeak_lower,Epeak_value,Epeak_upper
GRB,,,,,,,,,,
181222B,1,-0.028,-0.538,0.025,-0.143,-3.077,0.094,-9.0,365.0,10.0
181222B,2,-0.043,-0.118,0.045,-0.359,-4.145,0.292,-11.0,430.0,11.0
190206A,1,-0.085,-0.318,0.088,-0.544,-3.392,0.299,-102.0,1083.0,138.0
190305A,1,-0.063,-0.179,0.073,-0.143,-3.015,0.126,-22.0,435.0,20.0
190606A,1,-0.047,-1.020,0.047,-0.241,-2.120,0.154,-694.0,1672.0,905.0
200521A,1,-0.083,-0.392,0.087,-0.079,-2.200,0.089,-268.0,1411.0,156.0
200716C,1,-0.089,-0.537,0.099,-0.729,-2.830,0.300,-90.0,702.0,107.0
201227A$^{*}$,1,-0.067,-0.199,0.083,-0.896,-3.570,0.508,-68.0,948.0,52.0
210619B,1,-0.038,-0.733,0.047,-0.027,-2.217,0.028,-28.0,543.0,24.0


In [19]:
df = pd.DataFrame({
    'GRB': ['181222B', '||', '190206A', '||', '190305A', '190606A', '200521A', '200716C', '201227A', '210619B', '||'],
    'Int': [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2],
    'Model': ['Band', 'Band', 'Band', 'CPL', 'Band', 'Band', 'CPL', 'Band', 'CPL', 'Band', 'Band'],
    'alpha': [-0.520, -0.110, -0.501, 0.567, -0.162, -0.676, 0.259, -0.544, 0.190, -0.427, -0.500],
    'alpha_upper': [0.085, 0.126, 0.101, 0.091, 0.072, 0.493, 0.131, 0.208, 0.370, 0.122, 0.061],
    'alpha_lower': [-0.076, -0.112, -0.100, -0.092, -0.067, -0.296, -0.141, -0.163, -0.003, -0.109, -0.057],
    'beta': [-2.98, -3.53, -2.95, np.nan, -2.96, -1.69, np.nan, -2.31, np.nan, -2.06, -2.01],
    'beta_upper': [0.216, 0.410, 0.472, 1.0, 0.127, 0.145, 1.0, 0.284, 1.0, 0.090, 0.036],
    'beta_lower': [-0.314, -0.836, -2.950, -1.0, -0.158, -0.209, -1.0, -0.620, -1.0, -0.114, -0.039],
    'Epeak': [368, 417, 1402, 1593, 425, 478, 1333, 667, 882, 587, 433],
    'Epeak_upper': [28, 31, 366, 252, 21, 379, 195, 180, 138, 94, 30],
    'Epeak_lower': [-29, -32, -324, -595, -21, -226, -166, -153, -114, -78, -28]
})

df.loc[df['Model'] == 'CPL', ['beta', 'beta_upper', 'beta_lower']] = np.nan

df


,GRB,Int,Model,alpha,alpha_upper,alpha_lower,beta,beta_upper,beta_lower,Epeak,Epeak_upper,Epeak_lower
0,181222B,1,Band,-0.520,0.085,-0.076,-2.98,0.216,-0.314,368,28,-29
1,||,2,Band,-0.110,0.126,-0.112,-3.53,0.410,-0.836,417,31,-32
2,190206A,1,Band,-0.501,0.101,-0.100,-2.95,0.472,-2.950,1402,366,-324
3,||,1,CPL,0.567,0.091,-0.092,NaN,NaN,NaN,1593,252,-595
4,190305A,1,Band,-0.162,0.072,-0.067,-2.96,0.127,-0.158,425,21,-21
5,190606A,1,Band,-0.676,0.493,-0.296,-1.69,0.145,-0.209,478,379,-226
6,200521A,1,CPL,0.259,0.131,-0.141,NaN,NaN,NaN,1333,195,-166
7,200716C,1,Band,-0.544,0.208,-0.163,-2.31,0.284,-0.620,667,180,-153
8,201227A,1,CPL,0.190,0.370,-0.003,NaN,NaN,NaN,882,138,-114
9,210619B,1,Band,-0.427,0.122,-0.109,-2.06,0.090,-0.114,587,94,-78


In [21]:
df.to_csv('KW_modPar.csv',index=False)